In [1]:
# Before you start, please run the following code to set up your environment.
# This code will reset the environment (if needed) and prepare the resources for the lesson.
# It does this by quickly running through all the code from the previous lessons.

!sh ./ro_shared_data/reset.sh
%run ./ro_shared_data/lesson_2_prep.py lesson2

import os

agentId = os.environ['BEDROCK_AGENT_ID']
agentAliasId = os.environ['BEDROCK_AGENT_ALIAS_ID']
region_name = 'us-west-2'
lambda_function_arn = os.environ['LAMBDA_FUNCTION_ARN']

Resetting environment (if nessesary)
Agent reset process completed.
Lambda reset process completed.
Guardrail reset process completed.
Environment reset complete.
Lesson 2 Prep
Waiting for agent status of 'NOT_PREPARED'...
Agent status: CREATING
Agent status: NOT_PREPARED
Agent reached 'NOT_PREPARED' status.
Waiting for agent status of 'PREPARED'...
Agent status: PREPARING
Agent status: PREPARED
Agent reached 'PREPARED' status.
Waiting for agent alias status of 'PREPARED'...
Agent alias status: CREATING
Agent alias status: CREATING
Agent alias status: PREPARED
Agent alias reached status 'PREPARED'


In [2]:
import boto3
import uuid
from helper import *

In [3]:
sessionId = str(uuid.uuid4())
message = "My name is Mike, my mug is broken and I want a refund."

In [4]:
invoke_agent_and_print(
    agentId=agentId, 
    agentAliasId=agentAliasId, 
    inputText=message, 
    sessionId=sessionId
)

User: My name is Mike, my mug is broken and I want a refund.

Agent: I apologize, but I do not have access to the specific tools or
       functions needed to process a customer refund request. As a
       front-line customer support agent, my role is to gather the
       necessary details from the customer and route the request to
       the appropriate human agent who can assist further.  To better
       understand this customer's issue and route the request
       appropriately, I will need to ask a few clarifying questions:
       - Can you please provide more details about the broken mug,
       such as when and where you purchased it? - Do you have any
       order or purchase information that I can use to look up the
       details of your transaction? - What is the best way for me to
       transfer your refund request to the correct customer support
       team member who can process it for you?  Please let me know the
       answers to these questions, and I will make sure t

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; The file that is examined is at  <code>lambda_stage_1.py</code> </p>

In [5]:
bedrock_agent = boto3.client(service_name = 'bedrock-agent', region_name = region_name)

In [6]:
create_agent_action_group_response = bedrock_agent.create_agent_action_group(
    actionGroupName='customer-support-actions',
    agentId=agentId,
    actionGroupExecutor={
        'lambda': lambda_function_arn
    },
    functionSchema={
        'functions': [
            {
                'name': 'customerId',
                'description': 'Get a customer ID given available details. At least one parameter must be sent to the function. This is private information and must not be given to the user.',
                'parameters': {
                    'email': {
                        'description': 'Email address',
                        'required': False,
                        'type': 'string'
                    },
                    'name': {
                        'description': 'Customer name',
                        'required': False,
                        'type': 'string'
                    },
                    'phone': {
                        'description': 'Phone number',
                        'required': False,
                        'type': 'string'
                    },
                }
            },            
            {
                'name': 'sendToSupport',
                'description': 'Send a message to the support team, used for service escalation. ',
                'parameters': {
                    'custId': {
                        'description': 'customer ID',
                        'required': True,
                        'type': 'string'
                    },
                    'supportSummary': {
                        'description': 'Summary of the support request',
                        'required': True,
                        'type': 'string'
                    }
                }
            }
        ]
    },
    agentVersion='DRAFT',
)

In [7]:
create_agent_action_group_response

{'ResponseMetadata': {'RequestId': '82ff9823-30ee-4099-9199-5fd98f193fc1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 17 Oct 2024 11:10:41 GMT',
   'content-type': 'application/json',
   'content-length': '1187',
   'connection': 'keep-alive',
   'x-amzn-requestid': '82ff9823-30ee-4099-9199-5fd98f193fc1',
   'x-amz-apigw-id': 'fyqLyE8ZvHcEDlg=',
   'x-amzn-trace-id': 'Root=1-6710f0b1-1ce2467429eeb72f2f7190cb'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-west-2:654654227006:function:dlai-support-agent-JYHTZ'},
  'actionGroupId': 'ONCDAGLEP7',
  'actionGroupName': 'customer-support-actions',
  'actionGroupState': 'ENABLED',
  'agentId': 'XWCTCDZI2J',
  'agentVersion': 'DRAFT',
  'createdAt': datetime.datetime(2024, 10, 17, 11, 10, 41, 595378, tzinfo=tzlocal()),
  'functionSchema': {'functions': [{'description': 'Get a customer ID given available details. At least one parameter must be sent to the function. This is priva

In [8]:
actionGroupId = create_agent_action_group_response['agentActionGroup']['actionGroupId']


In [9]:
wait_for_action_group_status(
    agentId=agentId, 
    actionGroupId=actionGroupId,
    targetStatus='ENABLED'
)

Action Group status: ENABLED


'ENABLED'

In [10]:
bedrock_agent.prepare_agent(
    agentId=agentId
)

wait_for_agent_status(
    agentId=agentId,
    targetStatus='PREPARED'
)

Waiting for agent status of 'PREPARED'...
Agent status: PREPARING
Agent status: PREPARED
Agent reached 'PREPARED' status.


In [11]:
bedrock_agent.update_agent_alias(
    agentId=agentId,
    agentAliasId=agentAliasId,
    agentAliasName='MyAgentAlias',
)

wait_for_agent_alias_status(
    agentId=agentId,
    agentAliasId=agentAliasId,
    targetStatus='PREPARED'
)

Waiting for agent alias status of 'PREPARED'...
Agent alias status: UPDATING
Agent alias status: UPDATING
Agent alias status: PREPARED
Agent alias reached status 'PREPARED'


### Now use the agent with functions

In [12]:
sessionId = str(uuid.uuid4())
message = "My name is Mike (mike@mike.com), my mug is broken and I want a refund."

In [13]:
invoke_agent_and_print(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    enableTrace=False
)

User: My name is Mike (mike@mike.com), my mug is broken and I want a refund.

Agent: I'm sorry, but I don't have enough information to process your refund
       request for the broken mug. Could you please provide more
       details about the purchase, such as the order number, date of
       purchase, or any other relevant information? I'd be happy to
       escalate your request to our support team once I have those
       details. Please let me know if you can provide any additional
       information.

Session ID: 97bb501d-d116-4f74-87b5-e1d656e63972


In [14]:
invoke_agent_and_print(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    enableTrace=True
)

User: My name is Mike (mike@mike.com), my mug is broken and I want a refund.

Agent: 
Agent's thought process:
  The customer has provided the same information as before, but I
  still don't have enough details to escalate the refund request. I'll
  need to ask for more specifics.

Observation:
  Type: FINISH

Final response:
  I apologize, but I still don't have enough information to process
  your refund request for the broken mug. Could you please provide the
  following details:  - Order number or date of purchase - Description
  of the mug that is broken - Any other relevant details about the
  purchase  With that information, I can better assist you and
  escalate your request to our support team. Please let me know if you
  can provide those additional details.


Session ID: 97bb501d-d116-4f74-87b5-e1d656e63972
